In [2]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [3]:
url = 'https://www.genie.co.kr/chart/top200'
res = requests.get(url)
res.text

'<br>\r\n<br>\r\n<center>\r\n<br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [4]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers=header)
res.text[:500]

'\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n\t\n\n\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n<!--[if lte IE 6]><html dir="ltr" class="no-js ie6"><![endif]-->\n<!--[if lte IE 7]><html dir="ltr" class="no-js ie7"><![endif]-->\n<!--[if IE 8]><html dir="ltr" class="no-js ie8"><![endif]-->\n<!--[if IE 9]><html dir="ltr" class="no-js ie9"><![endif]-->\n<!--[if (gte IE 10)|!(IE)]><!--><html dir="ltr" class="no-js"><!--<![endif]-->\n<title>음악, 그리고 설레임 - 지니</title>\n<meta charset="utf-8'

In [5]:
soup = BeautifulSoup(res.text, 'html.parser')

In [6]:
trs = soup.select('.list-wrap > tbody > tr')
len(trs)

50

In [7]:
tr = trs[0]

In [8]:
rank = tr.select_one('.number').get_text().strip().replace('\n','')
rank = rank.replace(' ','')
rank,rank_none = rank[0],rank[1:]
rank, rank_none

('1', '유지')

In [9]:
def rank_date(s):
    if len(s) == 3:        # '2023.09.18.'
        return s[-2:]
    

In [10]:
album_cover = tr.select_one('.cover > img')['src']
album_cover

'//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/181/610/84181610_1692585955705_1_140x140.JPG/dims/resize/Q_80,0'

In [11]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'Love Lee'

In [12]:
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

'AKMU (악뮤)'

In [13]:
album_title = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album_title

'Love Lee'

In [14]:
from datetime import datetime, timedelta

now = datetime.now()
now

datetime.datetime(2023, 9, 19, 9, 13, 57, 390310)

In [19]:
date = now
date1 = f'{date.year}.{date.month:02d}.{date.day:02d}'
date1 = date1.replace('.','')
date1

'20230919'

In [16]:
date2 = f'{date.hour}'
date2

'9'

In [20]:
data = []
for page in tqdm(range(1, 5)):
    url = f'https://www.genie.co.kr/chart/top200?pg={page}'
    res = requests.get(url, headers=header)
    soup = BeautifulSoup(res.text, 'html.parser')
    trs = soup.select('.list-wrap > tbody > tr')
    
    for tr in trs:
        rank = tr.select_one('.number').get_text().strip().replace('\n','')
        rank = rank.replace(' ','')
        rank,rank_none = rank[0],rank[1:]
        rank, rank_none
        album_cover = tr.select_one('.cover > img')['src']
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album_title = tr.select_one('.albumtitle.ellipsis').get_text().strip()

        data.append({'순위':rank, '순위변동':rank_none, '앨범커버':album_cover, '제목':title, 
                    '가수':artist,'앨범타이틀':album_title})

100%|██████████| 4/4 [00:01<00:00,  3.10it/s]


In [21]:
df = pd.DataFrame(data)
df.head()

,순위,순위변동,앨범커버,제목,가수,앨범타이틀
0,1,유지,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/181/...,Love Lee,AKMU (악뮤),Love Lee
1,2,유지,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/193/...,Smoke (Prod. by Dynamicduo & Padi),다이나믹 듀오 & 이영지,스트릿 우먼 파이터2 (SWF2) 계급미션
2,3,유지,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/181/...,후라이의 꿈,AKMU (악뮤),Love Lee
3,4,유지,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/019/...,ETA,NewJeans,NewJeans 2nd EP 'Get Up'
4,5,1상승,//image.genie.co.kr/Y/IMAGE/IMG_ALBUM/084/048/...,Seven (Feat. Latto) (Clean Ver.),정국,Seven (Clean Ver.)
